In [18]:
from sklearn.datasets import load_iris
import pandas as pd


# Load the iris dataset
iris = load_iris()

# Create a DataFrame
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['target'] = iris.target

# Display the first few rows
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [19]:
# is my data balanced or imbalanced?
df.target.value_counts()

target
0    50
1    50
2    50
Name: count, dtype: int64

In [20]:
# Check for missing values in each column
missing_values = df.isnull().sum()
print("Missing values per column:")
print(missing_values)

# Total missing values in the dataset
total_missing = df.isnull().sum().sum()
print(f"\nTotal missing values: {total_missing}")

# Alternatively, get a summary of data types and non-null counts
df.info()

Missing values per column:
sepal length (cm)    0
sepal width (cm)     0
petal length (cm)    0
petal width (cm)     0
target               0
dtype: int64

Total missing values: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  150 non-null    float64
 1   sepal width (cm)   150 non-null    float64
 2   petal length (cm)  150 non-null    float64
 3   petal width (cm)   150 non-null    float64
 4   target             150 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 6.0 KB


In [21]:
df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [22]:
df.columns

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)', 'target'],
      dtype='object')

In [23]:
from sklearn.model_selection import train_test_split

# Define features (X) and target (y)
X = df.drop('target', axis=1)
y = df['target']

# First split: train and temp (temp will be split into validation and test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)

# Second split: validation and test from temp
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Check sizes
print(f"Train set: {X_train.shape[0]} samples")
print(f"Validation set: {X_val.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")


Train set: 90 samples
Validation set: 30 samples
Test set: 30 samples


In [24]:
import mlflow
import mlflow.sklearn
import shutil

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import os 

# Remove corrupted mlruns directory if it exists
#if os.path.exists("./mlruns"):
    #shutil.rmtree("./mlruns")

# Create mlruns directory
os.makedirs("./mlruns", exist_ok=True)
# Set up local tracking URI (creates a 'mlruns' folder in your current directory)
mlflow.set_tracking_uri("file:./mlruns")

# Create or set an experiment
mlflow.set_experiment("Iris Baseline Experiment")

solver = ["lbfgs", "newton-cg", "sag", "saga"]

for s in solver:
    # Start an MLflow run for each solver
    with mlflow.start_run():
        # Initialize and train the model
        model = LogisticRegression(random_state=42, max_iter=200, solver=s)
        model.fit(X_train, y_train)

        # Predict on validation set
        y_val_pred = model.predict(X_val)
        #print("Validation Accuracy:", accuracy_score(y_val, y_val_pred))
        #print("Validation Classification Report:")
        #print(classification_report(y_val, y_val_pred))

        # Predict on test set
        y_test_pred = model.predict(X_test)
        #print("Test Accuracy:", accuracy_score(y_test, y_test_pred))
        #print("Test Classification Report:")
        #print(classification_report(y_test, y_test_pred))
        
        # Log parameters
        mlflow.log_param("model_type", "LogisticRegression")
        mlflow.log_param("solver", s)
        mlflow.log_param("random_state", 42)
        mlflow.log_param("max_iter", 200)
        
        # Log metrics
        mlflow.log_metric("test_accuracy", accuracy_score(y_test, y_test_pred))
        mlflow.log_metric("test_precision_macro", classification_report(y_test, y_test_pred, output_dict=True)['macro avg']['precision'])
        mlflow.log_metric("test_recall_macro", classification_report(y_test, y_test_pred, output_dict=True)['macro avg']['recall'])
        mlflow.log_metric("test_f1_macro", classification_report(y_test, y_test_pred, output_dict=True)['macro avg']['f1-score'])
        
        # Log the model
        mlflow.sklearn.log_model(model, "model")
        
        # Get the run ID for registration
        run_id = mlflow.active_run().info.run_id

    # Register the model outside the run context
    mlflow.register_model(f"runs:/{run_id}/model", f"iris_baseline_{s}")

2026/02/20 12:59:46 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/Users/iashahba/Documents/ARTIFIN/FS26/SW2/Data and Code/.venv/lib/python3.14/site-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)
Registered model 'iris_baseline_lbfgs' already exists. Creating a new version of this model...
2026/02/20 12:59:49 WARNING mlflow.tracking._model_registry.fluent: Run with id d519c0e6133d4e6abf4788ff5f08b282 has no artifacts at artifact path 'model', registering model based on models:/m-747eda5106724622b37ebb854d6bbc40 instead
Created version '2' of model 'iris_baseline_lbfgs'.
2026/02/20 12:59:49 WARNING 

In [25]:
# After the loop, find the best run based on test_accuracy
best_runs = mlflow.search_runs(experiment_names=["Iris Baseline Experiment"], order_by=["metrics.test_accuracy DESC"])
best_run = best_runs.iloc[0]
best_run_id = best_run['run_id']
best_solver = best_run['params.solver']
best_accuracy = best_run['metrics.test_accuracy']

print(f"\nBest Model: {best_solver}")
print(f"Best Test Accuracy: {best_accuracy}")

# Register the best model as the baseline
mlflow.register_model(f"runs:/{best_run_id}/model", "iris_baseline")

print("Best model registered as 'iris_baseline'")

Registered model 'iris_baseline' already exists. Creating a new version of this model...
2026/02/20 12:59:56 WARNING mlflow.tracking._model_registry.fluent: Run with id 3c5089da30fe453fbace9a1b6aea6b56 has no artifacts at artifact path 'model', registering model based on models:/m-a44d65f42a0f4e54a4d05dc9c30409cd instead



Best Model: saga
Best Test Accuracy: 1.0
Best model registered as 'iris_baseline'


Created version '4' of model 'iris_baseline'.


In [26]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

model_version = 1
new_stage = "Staging"
client.transition_model_version_stage(
    name="iris_baseline",
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

print(f"Model 'iris_baseline' version {model_version} transitioned to {new_stage}")

Model 'iris_baseline' version 1 transitioned to Staging


/var/folders/kz/ws42_78n7g97qj91m287jwvh0000gp/T/ipykernel_8101/908629059.py:7: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


In [29]:
from datetime import datetime   
date = datetime.today().strftime("%Y-%m-%d")


# Update the model version description
client.update_model_version(
    name="iris_baseline",
    version=model_version,
    description=f"Best model from Iris Baseline Experiment with solver {best_solver} and test accuracy {best_accuracy:.4f} as of {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1771588285862, current_stage='Staging', deployment_job_state=None, description=('Best model from Iris Baseline Experiment with solver saga and test accuracy '
 '1.0000 as of 2026-02-20'), last_updated_timestamp=1771588871148, metrics=[<Metric: dataset_digest=None, dataset_name=None, key='test_f1_macro', model_id='m-90dcab2c6cd44e18b46b6cc7a23d0b35', run_id='c41d492b6c2245a2bb4272fa344c022f', step=0, timestamp=1771588177634, value=1.0>,
 <Metric: dataset_digest=None, dataset_name=None, key='test_accuracy', model_id='m-90dcab2c6cd44e18b46b6cc7a23d0b35', run_id='c41d492b6c2245a2bb4272fa344c022f', step=0, timestamp=1771588177625, value=1.0>,
 <Metric: dataset_digest=None, dataset_name=None, key='test_precision_macro', model_id='m-90dcab2c6cd44e18b46b6cc7a23d0b35', run_id='c41d492b6c2245a2bb4272fa344c022f', step=0, timestamp=1771588177628, value=1.0>,
 <Metric: dataset_digest=None, dataset_name=None, key='test_recall_macro', model_id='m-90dcab2c

In [27]:
from mlflow.tracking import MlflowClient
from datetime import datetime   

client = MlflowClient()
date = datetime.today().strftime("%Y-%m-%d")

# Use version 1 (the first registered version)
model_version = 1
new_stage = "Staging"

client.transition_model_version_stage(
    name="iris_baseline",
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

print(f"Model 'iris_baseline' version {model_version} transitioned to {new_stage}")

# Update the model version description
client.update_model_version(
    name="iris_baseline",
    version=model_version,
    description=f"Best model from Iris Baseline Experiment with solver {best_solver} and test accuracy {best_accuracy:.4f} as of {date}"
)


/var/folders/kz/ws42_78n7g97qj91m287jwvh0000gp/T/ipykernel_8101/1722267083.py:11: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


Model 'iris_baseline' version 1 transitioned to Staging


<ModelVersion: aliases=[], creation_timestamp=1771588285862, current_stage='Staging', deployment_job_state=None, description=('Best model from Iris Baseline Experiment with solver saga and test accuracy '
 '1.0000 as of 2026-02-20'), last_updated_timestamp=1771588796813, metrics=[<Metric: dataset_digest=None, dataset_name=None, key='test_f1_macro', model_id='m-90dcab2c6cd44e18b46b6cc7a23d0b35', run_id='c41d492b6c2245a2bb4272fa344c022f', step=0, timestamp=1771588177634, value=1.0>,
 <Metric: dataset_digest=None, dataset_name=None, key='test_accuracy', model_id='m-90dcab2c6cd44e18b46b6cc7a23d0b35', run_id='c41d492b6c2245a2bb4272fa344c022f', step=0, timestamp=1771588177625, value=1.0>,
 <Metric: dataset_digest=None, dataset_name=None, key='test_precision_macro', model_id='m-90dcab2c6cd44e18b46b6cc7a23d0b35', run_id='c41d492b6c2245a2bb4272fa344c022f', step=0, timestamp=1771588177628, value=1.0>,
 <Metric: dataset_digest=None, dataset_name=None, key='test_recall_macro', model_id='m-90dcab2c

In [28]:
import mlflow.pyfunc

# Load the best model from the model registry
model_uri = "models:/iris_baseline/Staging"
loaded_model = mlflow.pyfunc.load_model(model_uri)

# Make predictions on test set
predictions = loaded_model.predict(X_test)

# Display results
print("Predictions on test set:")
print(predictions[:10])  # First 10 predictions

# Evaluate
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test, predictions)
print(f"\nTest Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, predictions))

Predictions on test set:
[1 0 1 0 2 0 1 0 2 0]

Test Accuracy: 1.0000

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00        10

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

